In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


In [2]:
df = pd.read_csv('ds_clean/ds_clean_timeset.csv')
dfs = {location: group for location, group in df.groupby('LocationCode')}
for location, group in dfs.items():
    print(f"LocationCode: {location}")
    print(group)
    print()

LocationCode: 1
        LocationCode  WindSpeed(m/s)  Pressure(hpa)  Temperature(°C)  \
587855             1             0.0        1016.50             17.5   
587856             1             0.0        1016.53             17.5   
587857             1             0.0        1016.57             17.5   
587858             1             0.0        1016.58             17.5   
587859             1             0.0        1016.59             17.5   
...              ...             ...            ...              ...   
683763             1             0.0         959.80             25.1   
683764             1             0.0         959.92             25.1   
683765             1             0.0         960.09             25.1   
683766             1             0.0         959.40             25.1   
683767             1             0.0         959.02             25.0   

        Humidity(%)  Sunlight(Lux)  Power(mW)  DayOfYear  \
587855         86.8          25.00        0.0          1   

In [3]:
from datetime import datetime, timedelta
models = {}
for source in range(1,18):
    for destination in range(1,18):
        if source == destination:
            continue
        a = dfs[destination].copy()
        b = dfs[source].copy()
        base_year = 2024
        a['DateTime'] = a.apply(lambda row: datetime(base_year, 1, 1) + timedelta(days=int(row['DayOfYear'])-1, minutes=int(row['MinutesSinceStartOfDay'])), axis=1)
        b['DateTime'] = b.apply(lambda row: datetime(base_year, 1, 1) + timedelta(days=int(row['DayOfYear'])-1, minutes=int(row['MinutesSinceStartOfDay'])), axis=1)
        a['DateTime'] = pd.to_datetime(a['DateTime'])
        b['DateTime'] = pd.to_datetime(b['DateTime'])
        # 他有重複的資料(一分鐘內記了兩次)
        # print(a[a['IsDuplicate']==True])
        a = a.drop_duplicates(subset=['DateTime'])
        b = b.drop_duplicates(subset=['DateTime'])

        # print("Unique DateTime in a:", len(a['DateTime'].unique()))
        # print("Unique DateTime in b:", len(b['DateTime'].unique()))
        common_times = pd.merge(a, b, on='DateTime', how='inner')
        common_set = set(common_times['DateTime'])
        a_filtered = a[a['DateTime'].isin(common_set)]
        b_filtered = b[b['DateTime'].isin(common_set)]
        a_filtered = a_filtered.sort_values(by='DateTime').reset_index(drop=True)
        b_filtered = b_filtered.sort_values(by='DateTime').reset_index(drop=True)
        # print(a_filtered)
        # print(b_filtered)
        # print("Unique DateTime in a_filter:", len(a_filtered['DateTime'].unique()))
        # print("Unique DateTime in b_filter:", len(b_filtered['DateTime'].unique()))
        # print("Common DateTime in a and b:", len(common_times))
        # print("Duplicates in a['DateTime']:", a['DateTime'].duplicated().sum())
        # print("Duplicates in b['DateTime']:", b['DateTime'].duplicated().sum())
        # print("Duplicates in a_filtered['DateTime']:", a_filtered['DateTime'].duplicated().sum())
        # print("Duplicates in b_filtered['DateTime']:", b_filtered['DateTime'].duplicated().sum())
        # print("Duplicates in common_times['DateTime']:", common_times['DateTime'].duplicated().sum())
        y = a_filtered['Power(mW)']
        X = b_filtered.drop(columns=['DateTime'])
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        model = xgb.XGBRegressor()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        cumulative_mae = mae * len(y_test)
        print(f"cumulative_mae: {cumulative_mae}")
        models[(source, destination)] = model





cumulative_mae: 294394.36388204055
cumulative_mae: 244141.86230452353
cumulative_mae: 199119.66834462323
cumulative_mae: 240338.51358886913
cumulative_mae: 222888.46663281397
cumulative_mae: 177324.63128692537
cumulative_mae: 319699.64726632764
cumulative_mae: 186607.7123191114
cumulative_mae: 322348.3887674819
cumulative_mae: 65387.44762668634
cumulative_mae: 245113.00584445783
cumulative_mae: 281235.34717480245
cumulative_mae: 373544.75724798325
cumulative_mae: 565227.9914645373
cumulative_mae: 520127.2993620911
cumulative_mae: 682674.189630192
cumulative_mae: 278046.006078011
cumulative_mae: 145257.74979360914
cumulative_mae: 155568.13388372434
cumulative_mae: 187509.88766841346
cumulative_mae: 130492.7029117583
cumulative_mae: 66543.97623738484
cumulative_mae: 237063.97959225735
cumulative_mae: 179179.48953925338
cumulative_mae: 242445.57766685382
cumulative_mae: 60186.63269697556
cumulative_mae: 185821.53988867375
cumulative_mae: 278442.17659635167
cumulative_mae: 276218.941531814

In [4]:
day_minute_model = dfs.copy()
time_models = {}
for i in range(1,18):
    day_minute_model[i]  = day_minute_model[i][["MinutesSinceStartOfDay","DayOfYear","Power(mW)"]]
    day_minute_model[i] = day_minute_model[i].drop_duplicates(subset=['MinutesSinceStartOfDay','DayOfYear'])

for i in range(1,18):
    X = day_minute_model[i][["MinutesSinceStartOfDay","DayOfYear"]]
    y = day_minute_model[i][["Power(mW)"]]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    model = xgb.XGBRegressor()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    cumulative_mae = mae * len(y_test)
    print(f"cumulative_mae: {cumulative_mae}")
    time_models[i] = model


cumulative_mae: 1516420.323570965
cumulative_mae: 1262161.2910176152
cumulative_mae: 742026.0730977455
cumulative_mae: 673698.5749490535
cumulative_mae: 705016.816560562
cumulative_mae: 682927.0825222626
cumulative_mae: 453159.21923047485
cumulative_mae: 938742.4362818906
cumulative_mae: 502476.03142640425
cumulative_mae: 908562.8549131765
cumulative_mae: 149170.1354569563
cumulative_mae: 614451.1733248897
cumulative_mae: 674080.8172726127
cumulative_mae: 837733.3824745021
cumulative_mae: 1078529.3718543812
cumulative_mae: 896378.4181768288
cumulative_mae: 1297637.0745270795


In [21]:
base_year = 2024
second_layer_models = {}
for destination in range(1,18):
    destination_unique_time = dfs[destination].drop_duplicates(subset=['MinutesSinceStartOfDay','DayOfYear']).copy()
    destination_unique_time['DateTime'] = destination_unique_time.apply(lambda row: datetime(base_year, 1, 1) + timedelta(days=int(row['DayOfYear'])-1, minutes=int(row['MinutesSinceStartOfDay'])), axis=1)
    unique_timeset = set(destination_unique_time['DateTime'])
    merged_rest = []
    for source in [k for k in range(1, 18) if k != destination]:
        value = dfs[source].copy()
        value['DateTime'] = value.apply(lambda row: datetime(base_year, 1, 1) + timedelta(days=int(row['DayOfYear'])-1, minutes=int(row['MinutesSinceStartOfDay'])), axis=1)
        value.drop_duplicates(subset=['DateTime'], inplace=True)
        merged_rest.append(value)

    merged_rest_dfs = pd.concat(merged_rest, ignore_index=True)
    merged_rest_dfs['DateTime'] = merged_rest_dfs.apply(lambda row: datetime(base_year, 1, 1) + timedelta(days=int(row['DayOfYear'])-1, minutes=int(row['MinutesSinceStartOfDay'])), axis=1)
    merged_rest_dfs['anticipated_power'] = None

    for source in [k for k in range(1, 18) if k != destination]:
        subset = merged_rest_dfs[merged_rest_dfs['LocationCode'] == source]
        features = subset.drop(columns=['DateTime', 'anticipated_power'])
        predictions = models[(source, destination)].predict(features)
        predictions = predictions.astype(float)
        merged_rest_dfs.loc[merged_rest_dfs['LocationCode'] == source, 'anticipated_power'] = predictions

    merged_rest_dfs['anticipated_power'] = pd.to_numeric(merged_rest_dfs['anticipated_power'], errors='coerce')
    second_train = merged_rest_dfs[['DayOfYear', 'MinutesSinceStartOfDay', 'DateTime', 'LocationCode', 'anticipated_power']]

    # 將資料 pivot，使每個 LocationCode 都會有自己的 anticipated_power 欄位
    second_train_pivot = second_train.pivot(index=['DayOfYear', 'MinutesSinceStartOfDay', 'DateTime'], 
                        columns='LocationCode', values='anticipated_power')

    # 將欄位名稱改成想要的格式，如 LocationXAnticipated
    second_train_pivot.columns = [f"Location{int(col)}Anticipated" for col in second_train_pivot.columns]

    # 將原本的複合索引重設為欄位，這樣 DayOfYear、MinutesSinceStartOfDay、DateTime 都會變成普通的 columns
    second_train_pivot = second_train_pivot.reset_index()
    self_features = second_train_pivot[[ 'MinutesSinceStartOfDay','DayOfYear']]
    self_predictions = time_models[destination].predict(self_features)
    second_train_pivot[f'Location{destination}Anticipated'] = self_predictions
    
    
    second_train_merged = pd.merge(destination_unique_time, second_train_pivot, on=['DateTime'], how='inner')
    second_train_merged.rename(columns={'Power(mW)': f'Location{destination}Actual'}, inplace=True)
    print(second_train_merged.columns)
    X = second_train_merged[['Location1Anticipated', 'Location2Anticipated', 'Location3Anticipated', 'Location4Anticipated', 'Location5Anticipated', 'Location6Anticipated', 'Location7Anticipated', 'Location8Anticipated', 'Location9Anticipated', 'Location10Anticipated', 'Location11Anticipated', 'Location12Anticipated', 'Location13Anticipated', 'Location14Anticipated', 'Location15Anticipated', 'Location16Anticipated', 'Location17Anticipated']]
    print(X.dtypes)
    y = second_train_merged[f'Location{destination}Actual']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    model = xgb.XGBRegressor(missing=np.nan, enable_categorical=True)
    print(X_train.dtypes)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    cumulative_mae = mae * len(y_test)
    print(f"cumulative_mae: {cumulative_mae} ")
    print(f"mae: {mae}")
    second_layer_models[destination] = model

# a_locaiton_unique_time = dfs[1].drop_duplicates(subset=['MinutesSinceStartOfDay','DayOfYear']).copy()
# a_locaiton_unique_time['DateTime'] = a_locaiton_unique_time.apply(lambda row: datetime(base_year, 1, 1) + timedelta(days=int(row['DayOfYear'])-1, minutes=int(row['MinutesSinceStartOfDay'])), axis=1)
# unique_timeset = set(a_locaiton_unique_time['DateTime'])
# merged_rest = []
# for key in [k for k in range(2, 18) if k in dfs]:
#     value = dfs[key].copy()
#     value['DateTime'] = value.apply(lambda row: datetime(base_year, 1, 1) + timedelta(days=int(row['DayOfYear'])-1, minutes=int(row['MinutesSinceStartOfDay'])), axis=1)
#     value.drop_duplicates(subset=['DateTime'], inplace=True)
#     merged_rest.append(value)

# merged_rest_dfs = pd.concat(merged_rest, ignore_index=True)
    

# merged_rest_dfs['DateTime'] = merged_rest_dfs.apply(lambda row: datetime(base_year, 1, 1) + timedelta(days=int(row['DayOfYear'])-1, minutes=int(row['MinutesSinceStartOfDay'])), axis=1)
# merged_rest_dfs['anticipated_power'] = None
# merged_rest_dfs

Index(['LocationCode', 'WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)',
       'Humidity(%)', 'Sunlight(Lux)', 'Location1Actual', 'DayOfYear_x',
       'MinutesSinceStartOfDay_x', 'DateTime', 'DayOfYear_y',
       'MinutesSinceStartOfDay_y', 'Location2Anticipated',
       'Location3Anticipated', 'Location4Anticipated', 'Location5Anticipated',
       'Location6Anticipated', 'Location7Anticipated', 'Location8Anticipated',
       'Location9Anticipated', 'Location10Anticipated',
       'Location11Anticipated', 'Location12Anticipated',
       'Location13Anticipated', 'Location14Anticipated',
       'Location15Anticipated', 'Location16Anticipated',
       'Location17Anticipated', 'Location1Anticipated'],
      dtype='object')
Location1Anticipated     float32
Location2Anticipated     float64
Location3Anticipated     float64
Location4Anticipated     float64
Location5Anticipated     float64
Location6Anticipated     float64
Location7Anticipated     float64
Location8Anticipated     float64
L

In [6]:
# for i in range(2, 18):
#     subset = merged_rest_dfs[merged_rest_dfs['LocationCode'] == i]
#     features = subset.drop(columns=['DateTime', 'anticipated_power'])
#     predictions = models[(i, 1)].predict(features)
#     predictions = predictions.astype(float)
#     merged_rest_dfs.loc[merged_rest_dfs['LocationCode'] == i, 'anticipated_power'] = predictions

# merged_rest_dfs['anticipated_power'] = pd.to_numeric(merged_rest_dfs['anticipated_power'], errors='coerce')

# merged_rest_dfs


In [7]:
# second_train = merged_rest_dfs[['DayOfYear', 'MinutesSinceStartOfDay', 'DateTime', 'LocationCode', 'anticipated_power']]

# # 將資料 pivot，使每個 LocationCode 都會有自己的 anticipated_power 欄位
# second_train_pivot = second_train.pivot(index=['DayOfYear', 'MinutesSinceStartOfDay', 'DateTime'], 
#                     columns='LocationCode', values='anticipated_power')

# # 將欄位名稱改成想要的格式，如 LocationXAnticipated
# second_train_pivot.columns = [f"Location{int(col)}Anticipated" for col in second_train_pivot.columns]

# # 將原本的複合索引重設為欄位，這樣 DayOfYear、MinutesSinceStartOfDay、DateTime 都會變成普通的 columns
# second_train_pivot = second_train_pivot.reset_index()

# # 現在 df_pivot 裡面您就有：
# # DayOfYear, MinutesSinceStartOfDay, DateTime (作為特徵欄位)
# # 以及 Location1Anticipated, Location2Anticipated, ... 等欄位

# # second_train_pivot.drop(columns=['DateTime'], inplace=True)
# second_train_pivot


In [8]:
# self_features = second_train_pivot[[ 'MinutesSinceStartOfDay','DayOfYear']]
# self_predictions = time_models[1].predict(self_features)
# second_train_pivot['Location1Anticipated'] = self_predictions
# second_train_pivot

In [9]:
# second_train_pivot.dtypes

In [10]:
# second_train_merged = pd.merge(a_locaiton_unique_time, second_train_pivot, on=['DateTime'], how='inner')
# second_train_merged.rename(columns={'Power(mW)': 'Location1Actual'}, inplace=True)
# X = second_train_merged.drop(columns=['DateTime', 'Location1Actual'])
# y = second_train_merged['Location1Actual']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# model = xgb.XGBRegressor(missing=np.nan, enable_categorical=True)
# print(X_train.dtypes)
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# mae = mean_absolute_error(y_test, y_pred)
# cumulative_mae = mae * len(y_test)
# print(f"mae: {mae}")


## 驗證

In [11]:
question = pd.read_csv('ds_clean/upload.csv')
# 将 '序號' 列转换为字符串
question['序號'] = question['序號'].astype(str)

# 提取 DayOfYear, MinutesSinceStartOfDay 和 LocationCode
question['DayOfYear'] = pd.to_datetime(question['序號'].str[:8], format='%Y%m%d').dt.dayofyear
question['MinutesSinceStartOfDay'] = question['序號'].str[8:12].apply(lambda x: int(x[:2]) * 60 + int(x[2:4]))
question['LocationCode'] = question['序號'].str[12:14].astype(int)

# 删除原来的 '序號' 列
question = question.drop(columns=['序號'])
question 

,答案,DayOfYear,MinutesSinceStartOfDay,LocationCode
0,NaN,1,540,1
1,NaN,1,550,1
2,NaN,1,560,1
3,NaN,1,570,1
4,NaN,1,580,1
...,...,...,...,...
9595,NaN,193,970,17
9596,NaN,193,980,17
9597,NaN,193,990,17
9598,NaN,193,1000,17


In [12]:
for i in range(1, 18):
    question[f"Location{i}Anticipated"] = None

question


merged = question.merge(df, on=['DayOfYear', 'MinutesSinceStartOfDay'], how='left')
merged

,答案,DayOfYear,MinutesSinceStartOfDay,LocationCode_x,Location1Anticipated,Location2Anticipated,Location3Anticipated,Location4Anticipated,Location5Anticipated,Location6Anticipated,...,Location15Anticipated,Location16Anticipated,Location17Anticipated,LocationCode_y,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW)
0,NaN,1,540,1,None,None,None,None,None,None,...,None,None,None,17.0,0.00,1024.43,19.00,65.30,5677.50,7.08
1,NaN,1,550,1,None,None,None,None,None,None,...,None,None,None,17.0,0.00,1024.43,19.00,65.30,5152.50,5.97
2,NaN,1,560,1,None,None,None,None,None,None,...,None,None,None,17.0,0.00,1024.47,19.10,64.90,7272.50,11.50
3,NaN,1,570,1,None,None,None,None,None,None,...,None,None,None,17.0,0.00,1024.68,19.20,64.90,8965.00,16.63
4,NaN,1,580,1,None,None,None,None,None,None,...,None,None,None,17.0,0.00,1024.59,19.30,64.90,9902.50,20.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70002,NaN,193,1010,17,None,None,None,None,None,None,...,None,None,None,2.0,0.00,1000.02,36.19,52.49,7743.33,18.11
70003,NaN,193,1010,17,None,None,None,None,None,None,...,None,None,None,5.0,0.61,999.80,37.03,57.64,11121.67,24.85
70004,NaN,193,1010,17,None,None,None,None,None,None,...,None,None,None,6.0,0.00,999.32,38.18,47.47,9539.17,25.34
70005,NaN,193,1010,17,None,None,None,None,None,None,...,None,None,None,8.0,0.00,1000.20,34.53,75.54,4860.83,5.48


In [13]:
import ipdb

if 'LocationCode_x' in merged.columns and 'LocationCode_y' in merged.columns:
    # 重命名以避免混淆
    merged.rename(columns={'LocationCode_x': 'LocationCode_from', 'LocationCode_y': 'LocationCode_to'}, inplace=True)

# 確保特徵欄位存在
expected_columns = [
    'LocationCode_to',
    'WindSpeed(m/s)',
    'Pressure(hpa)',
    'Temperature(°C)',
    'Humidity(%)',
    'Sunlight(Lux)',
    'Power(mW)',
    'DayOfYear',
    'MinutesSinceStartOfDay'
]

# 處理 merged 的數值型資料
merged_numeric = merged.drop(columns=['DateTime', 'anticipated_power', '答案'], errors='ignore')
merged_numeric = merged_numeric.apply(pd.to_numeric, errors='coerce')
merged_numeric = merged_numeric.drop(
    columns=[col for col in merged_numeric.columns if col.startswith('Location') and col.endswith('Anticipated')],
    errors='ignore'
)


# 確保 merged 的數值型資料包含預期欄位
for col in expected_columns:
    if col not in merged_numeric.columns:
        raise ValueError(f"缺少必要的特徵欄位: {col}")

# 初始化所有預測欄位
for i in range(1, 18):
    merged[f'Location{i}Anticipated'] = None
print(merged_numeric.columns)
# 基於 LocationCode 處理資料
for location_from in merged_numeric['LocationCode_from'].unique():
    for location_to in question['LocationCode'].unique():
        if location_from == location_to:
            continue  # 略過相同的 LocationCode

        # 提取來源特徵資料，將 LocationCode_from 改為 LocationCode 僅限於 feats 中
      
        feats = merged_numeric[(merged_numeric['LocationCode_from'] == location_from) & (merged_numeric['LocationCode_to'] == location_to)].copy()
        feats = feats[expected_columns]
    
        feats.rename(columns={'LocationCode_to': 'LocationCode'}, inplace=True)


        # 檢查模型是否存在
        model_key = (location_to, location_from)
        if model_key not in models:
            continue  # 跳過無模型的組合

        # 使用模型進行預測
        predictions = models[model_key].predict(feats)

        # 填入對應欄位
        merged.loc[(merged_numeric['LocationCode_from'] == location_from) & (merged_numeric['LocationCode_to'] == location_to), f'Location{location_to}Anticipated'] = predictions.astype(float)

# 將結果回寫到 question


# 最終結果
print(merged.columns)
print(merged.shape)
merged




Index(['DayOfYear', 'MinutesSinceStartOfDay', 'LocationCode_from',
       'LocationCode_to', 'WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)',
       'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)'],
      dtype='object')
Index(['答案', 'DayOfYear', 'MinutesSinceStartOfDay', 'LocationCode_from',
       'Location1Anticipated', 'Location2Anticipated', 'Location3Anticipated',
       'Location4Anticipated', 'Location5Anticipated', 'Location6Anticipated',
       'Location7Anticipated', 'Location8Anticipated', 'Location9Anticipated',
       'Location10Anticipated', 'Location11Anticipated',
       'Location12Anticipated', 'Location13Anticipated',
       'Location14Anticipated', 'Location15Anticipated',
       'Location16Anticipated', 'Location17Anticipated', 'LocationCode_to',
       'WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)',
       'Sunlight(Lux)', 'Power(mW)'],
      dtype='object')
(70007, 28)


,答案,DayOfYear,MinutesSinceStartOfDay,LocationCode_from,Location1Anticipated,Location2Anticipated,Location3Anticipated,Location4Anticipated,Location5Anticipated,Location6Anticipated,...,Location15Anticipated,Location16Anticipated,Location17Anticipated,LocationCode_to,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW)
0,NaN,1,540,1,None,None,None,None,None,None,...,None,None,127.530617,17.0,0.00,1024.43,19.00,65.30,5677.50,7.08
1,NaN,1,550,1,None,None,None,None,None,None,...,None,None,34.25071,17.0,0.00,1024.43,19.00,65.30,5152.50,5.97
2,NaN,1,560,1,None,None,None,None,None,None,...,None,None,118.955971,17.0,0.00,1024.47,19.10,64.90,7272.50,11.50
3,NaN,1,570,1,None,None,None,None,None,None,...,None,None,385.947052,17.0,0.00,1024.68,19.20,64.90,8965.00,16.63
4,NaN,1,580,1,None,None,None,None,None,None,...,None,None,345.716705,17.0,0.00,1024.59,19.30,64.90,9902.50,20.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70002,NaN,193,1010,17,None,35.409573,None,None,None,None,...,None,None,None,2.0,0.00,1000.02,36.19,52.49,7743.33,18.11
70003,NaN,193,1010,17,None,None,None,None,23.11949,None,...,None,None,None,5.0,0.61,999.80,37.03,57.64,11121.67,24.85
70004,NaN,193,1010,17,None,None,None,None,None,60.39957,...,None,None,None,6.0,0.00,999.32,38.18,47.47,9539.17,25.34
70005,NaN,193,1010,17,None,None,None,None,None,None,...,None,None,None,8.0,0.00,1000.20,34.53,75.54,4860.83,5.48


In [14]:
merged.drop(columns=['LocationCode_to','WindSpeed(m/s)','Pressure(hpa)',	'Temperature(°C)',	'Humidity(%)',	'Sunlight(Lux)',	'Power(mW)'], inplace=True)
merged


,答案,DayOfYear,MinutesSinceStartOfDay,LocationCode_from,Location1Anticipated,Location2Anticipated,Location3Anticipated,Location4Anticipated,Location5Anticipated,Location6Anticipated,...,Location8Anticipated,Location9Anticipated,Location10Anticipated,Location11Anticipated,Location12Anticipated,Location13Anticipated,Location14Anticipated,Location15Anticipated,Location16Anticipated,Location17Anticipated
0,NaN,1,540,1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,127.530617
1,NaN,1,550,1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,34.25071
2,NaN,1,560,1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,118.955971
3,NaN,1,570,1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,385.947052
4,NaN,1,580,1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,345.716705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70002,NaN,193,1010,17,None,35.409573,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
70003,NaN,193,1010,17,None,None,None,None,23.11949,None,...,None,None,None,None,None,None,None,None,None,None
70004,NaN,193,1010,17,None,None,None,None,None,60.39957,...,None,None,None,None,None,None,None,None,None,None
70005,NaN,193,1010,17,None,None,None,None,None,None,...,26.875599,None,None,None,None,None,None,None,None,None


In [ ]:
for i in range(1,18):
    self_predictions_subset = question[question['LocationCode'] == i][[ 'MinutesSinceStartOfDay','DayOfYear']]
    self_predictions = time_models[i].predict(self_predictions_subset)
    question.loc[question['LocationCode'] == i, f'Location{i}Anticipated'] = self_predictions

    question

,答案,DayOfYear,MinutesSinceStartOfDay,LocationCode,Location1Anticipated,Location2Anticipated,Location3Anticipated,Location4Anticipated,Location5Anticipated,Location6Anticipated,...,Location8Anticipated,Location9Anticipated,Location10Anticipated,Location11Anticipated,Location12Anticipated,Location13Anticipated,Location14Anticipated,Location15Anticipated,Location16Anticipated,Location17Anticipated
0,NaN,1,540,1,558.919067,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,NaN,1,550,1,669.169617,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,NaN,1,560,1,1117.860596,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,NaN,1,570,1,1188.838135,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,NaN,1,580,1,1334.312256,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9595,NaN,193,970,17,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,9.838187
9596,NaN,193,980,17,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,6.937551
9597,NaN,193,990,17,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,5.308323
9598,NaN,193,1000,17,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,5.308323


In [17]:
merged = merged.rename(columns={"LocationCode_from": "LocationCode"})

# 2. 合併數據：按鍵值合併
keys = ['DayOfYear', 'MinutesSinceStartOfDay', 'LocationCode']

# 合併兩個 DataFrame，外聯接保證所有鍵值完整
combined = pd.merge(question, merged, on=keys, suffixes=('_question', '_merged'), how='outer')

# 3. 數值欄位融合
# 對 `LocationXAnticipated` 欄位逐列進行融合處理
location_cols = [col for col in combined.columns if "Location" in col and "Anticipated" in col]

fused_data = pd.DataFrame()
for col in location_cols:
    # 將 `question` 和 `merged` 的列合併，優先保留 `question` 的值
    if '_question' in col:
        base_col = col.replace('_question', '')
        question_col = col
        merged_col = base_col + '_merged'
        fused_data[base_col] = combined[question_col].combine_first(combined[merged_col])

# 4. 保留不參與融合的欄位（包括鍵值和 `答案`）
result = pd.concat([combined[keys], fused_data], axis=1)
result['答案'] = combined['答案_question']  # 保持 `答案` 欄位原樣

# 5. 檢查鍵值是否缺失，若缺失則報錯
if result[keys].isnull().any().any():
    raise ValueError("合併後發現鍵值缺失，請檢查數據來源。")

# 6. 確保數據唯一性，對鍵值去重
result = result.drop_duplicates(subset=keys)
result

,DayOfYear,MinutesSinceStartOfDay,LocationCode,Location1Anticipated,Location2Anticipated,Location3Anticipated,Location4Anticipated,Location5Anticipated,Location6Anticipated,Location7Anticipated,...,Location9Anticipated,Location10Anticipated,Location11Anticipated,Location12Anticipated,Location13Anticipated,Location14Anticipated,Location15Anticipated,Location16Anticipated,Location17Anticipated,答案
0,1,540,1,558.919067,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,127.530617,NaN
1,1,550,1,669.169617,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,34.25071,NaN
2,1,560,1,1117.860596,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,118.955971,NaN
3,1,570,1,1188.838135,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,385.947052,NaN
4,1,580,1,1334.312256,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,345.716705,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69982,203,970,7,None,None,None,None,None,None,49.998367,...,None,-69.706429,None,None,None,None,None,None,None,NaN
69987,203,980,7,None,None,None,None,None,None,38.31118,...,None,-28.291557,None,None,None,None,None,None,None,NaN
69992,203,990,7,None,None,None,None,None,None,38.622398,...,None,-53.597542,None,None,None,None,None,None,None,NaN
69997,203,1000,7,None,None,None,None,None,None,29.207642,...,None,-13.941762,None,None,None,None,None,None,None,NaN


In [24]:
for i in range(1,18):
    subset = result[result['LocationCode'] == i]
    features = subset.drop(columns=['DayOfYear', 'MinutesSinceStartOfDay', 'LocationCode', '答案'])
    features = features.apply(lambda col: pd.to_numeric(col, errors='coerce') if col.dtype == 'object' else col)
    predictions = second_layer_models[i].predict(features)
    result.loc[result['LocationCode'] == i, '答案'] = predictions

result

,DayOfYear,MinutesSinceStartOfDay,LocationCode,Location1Anticipated,Location2Anticipated,Location3Anticipated,Location4Anticipated,Location5Anticipated,Location6Anticipated,Location7Anticipated,...,Location9Anticipated,Location10Anticipated,Location11Anticipated,Location12Anticipated,Location13Anticipated,Location14Anticipated,Location15Anticipated,Location16Anticipated,Location17Anticipated,答案
0,1,540,1,558.919067,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,127.530617,324.234222
1,1,550,1,669.169617,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,34.25071,458.043518
2,1,560,1,1117.860596,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,118.955971,587.771973
3,1,570,1,1188.838135,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,385.947052,713.345276
4,1,580,1,1334.312256,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,345.716705,477.877319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69982,203,970,7,None,None,None,None,None,None,49.998367,...,None,-69.706429,None,None,None,None,None,None,None,32.443695
69987,203,980,7,None,None,None,None,None,None,38.31118,...,None,-28.291557,None,None,None,None,None,None,None,27.607430
69992,203,990,7,None,None,None,None,None,None,38.622398,...,None,-53.597542,None,None,None,None,None,None,None,27.607430
69997,203,1000,7,None,None,None,None,None,None,29.207642,...,None,-13.941762,None,None,None,None,None,None,None,12.766551


寫回序列

In [27]:
final_result = result[['DayOfYear', 'MinutesSinceStartOfDay', 'LocationCode', '答案']].copy()
final_result["Date"] = final_result["DayOfYear"].apply(lambda x: (datetime(2024, 1, 1) + timedelta(days=x-1)).strftime("%Y%m%d"))
final_result["HHMM"] = final_result["MinutesSinceStartOfDay"].apply(lambda x: f"{x // 60:02}{x % 60:02}")
final_result["LocationCodeStr"] = final_result["LocationCode"].apply(lambda x: f"{x:02}")
final_result["序號"] = final_result["Date"] + final_result["HHMM"] + final_result["LocationCodeStr"]
final_result


,DayOfYear,MinutesSinceStartOfDay,LocationCode,答案,Date,HHMM,LocationCodeStr,序號
0,1,540,1,324.234222,20240101,0900,01,20240101090001
1,1,550,1,458.043518,20240101,0910,01,20240101091001
2,1,560,1,587.771973,20240101,0920,01,20240101092001
3,1,570,1,713.345276,20240101,0930,01,20240101093001
4,1,580,1,477.877319,20240101,0940,01,20240101094001
...,...,...,...,...,...,...,...,...
69982,203,970,7,32.443695,20240721,1610,07,20240721161007
69987,203,980,7,27.607430,20240721,1620,07,20240721162007
69992,203,990,7,27.607430,20240721,1630,07,20240721163007
69997,203,1000,7,12.766551,20240721,1640,07,20240721164007


In [29]:
final_result = final_result.sort_values(
    by=["LocationCode", "Date", "HHMM"],  # 按照多個列排序
    key=lambda col: col.astype(int),      # 將所有列轉換為 int 後排序
    ascending=True                        # 遞增排序
).reset_index(drop=True)                  # 重設索引
final_result


,DayOfYear,MinutesSinceStartOfDay,LocationCode,答案,Date,HHMM,LocationCodeStr,序號
0,1,540,1,324.234222,20240101,0900,01,20240101090001
1,1,550,1,458.043518,20240101,0910,01,20240101091001
2,1,560,1,587.771973,20240101,0920,01,20240101092001
3,1,570,1,713.345276,20240101,0930,01,20240101093001
4,1,580,1,477.877319,20240101,0940,01,20240101094001
...,...,...,...,...,...,...,...,...
9595,193,970,17,6.852645,20240711,1610,17,20240711161017
9596,193,980,17,6.852645,20240711,1620,17,20240711162017
9597,193,990,17,6.852645,20240711,1630,17,20240711163017
9598,193,1000,17,6.852645,20240711,1640,17,20240711164017


In [30]:
upload_df = pd.DataFrame()
upload_df = final_result[['序號', '答案']]

In [34]:
upload_df.to_csv('upload.csv', index=False)

In [33]:
import zipfile
with zipfile.ZipFile("upload.zip", 'w') as zf:
    zf.write("ds_clean/upload.csv")